In [122]:
import pandas as pd

moonpay_addr="0x151b381058f91cf871e7ea1ee83c45326f61e96d"
w0_addr="0xa1c7fb8cb261aE26C0aEFCD9edEcba7860bA69c2"
w1_addr="0xc6108f89578c8394dda250626d63f9a334a872fa"

data_path="../TransactionsData/csv/"
w0_path=data_path+"wallet-0.csv"
w1_path=data_path+"wallet-1.csv"
w0=pd.read_csv(w0_path).rename(columns={"CurrentValue @ $1647.22/Eth": "ValueAtDownloadTime"})
w1=pd.read_csv(w1_path).rename(columns={"CurrentValue @ $1672.43/Eth": "ValueAtDownloadTime"})

print(w0.shape)
print(w1.shape)
print((w0.columns.values==w1.columns.values).all())

w0.tail()

(116, 17)
(184, 17)
True


,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),ValueAtDownloadTime,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method,PrivateNote
111,0xe2009a45c044bba3e559ca3fba32df5fcbef9a347a39...,16403966,1673686295,2023-01-14 08:51:35,0x6ec88a2cb932eb46dfda0280c0eadb93b6eca13b,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.633001,0.000000,1042.691166,0.000352,0.579364,1550.07,NaN,NaN,Transfer,NaN
112,0x7454e4643dcbda3b88e2b2b833c84d97c6340c09d833...,16403979,1673686451,2023-01-14 08:54:11,0x6ec88a2cb932eb46dfda0280c0eadb93b6eca13b,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.633513,0.000000,1043.535745,0.000403,0.663872,1550.07,NaN,NaN,Transfer,NaN
113,0x592d28fdd58c27a8b5ff47ebdc380aa8e941803ed48f...,16404053,1673687339,2023-01-14 09:08:59,0x8c8d7c46219d9205f056f28fee5950ad564d7465,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.672233,0.000000,1107.315348,0.000398,0.655825,1550.07,NaN,NaN,Transfer,NaN
114,0x26277c1adb3f81affc78696fcf35a27298dcc53cf529...,16405002,1673698787,2023-01-14 12:19:47,0x8216874887415e2650d12d53ff53516f04a74fd7,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.206100,0.000000,339.492042,0.000420,0.691704,1550.07,NaN,NaN,Transfer,NaN
115,0xab0cf426b4879324640b2fc41d86fbbaa240e1a87fa5...,16407586,1673729939,2023-01-14 20:58:59,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,0x993f36f6bd7e92e1dc36faaecbe700df3534a34a,NaN,0.000000,3.096342,5100.356744,0.000395,0.649967,1550.07,NaN,NaN,Transfer,NaN


In [123]:
def add_ValueAtTime_col(df):
    df_IN = df[df["Value_OUT(ETH)"]==0.0]
    valueAtTimeIn = df_IN["Value_IN(ETH)"] * df_IN["Historical $Price/Eth"]
    df_IN = df_IN.assign(ValueAtTxTimeUSD=valueAtTimeIn)

    df_OUT=df[df["Value_OUT(ETH)"]!=0.0]
    valueAtTimeOut = -df_OUT["Value_OUT(ETH)"] * df_OUT["Historical $Price/Eth"]
    df_OUT = df_OUT.assign(ValueAtTxTimeUSD=valueAtTimeOut)

    res=pd.concat([df_IN, df_OUT], axis=0,ignore_index=True).sort_values(by=["DateTime"])
    res.reset_index(inplace=True,drop=True)
    return res

In [126]:
w0 = add_ValueAtTime_col(w0)
w1 = add_ValueAtTime_col(w1)

w0.tail()

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),ValueAtDownloadTime,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method,PrivateNote,ValueAtTxTimeUSD
111,0xe2009a45c044bba3e559ca3fba32df5fcbef9a347a39...,16403966,1673686295,2023-01-14 08:51:35,0x6ec88a2cb932eb46dfda0280c0eadb93b6eca13b,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.633001,0.000000,1042.691166,0.000352,0.579364,1550.07,NaN,NaN,Transfer,NaN,981.195163
112,0x7454e4643dcbda3b88e2b2b833c84d97c6340c09d833...,16403979,1673686451,2023-01-14 08:54:11,0x6ec88a2cb932eb46dfda0280c0eadb93b6eca13b,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.633513,0.000000,1043.535745,0.000403,0.663872,1550.07,NaN,NaN,Transfer,NaN,981.989930
113,0x592d28fdd58c27a8b5ff47ebdc380aa8e941803ed48f...,16404053,1673687339,2023-01-14 09:08:59,0x8c8d7c46219d9205f056f28fee5950ad564d7465,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.672233,0.000000,1107.315348,0.000398,0.655825,1550.07,NaN,NaN,Transfer,NaN,1042.007929
114,0x26277c1adb3f81affc78696fcf35a27298dcc53cf529...,16405002,1673698787,2023-01-14 12:19:47,0x8216874887415e2650d12d53ff53516f04a74fd7,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,NaN,0.206100,0.000000,339.492042,0.000420,0.691704,1550.07,NaN,NaN,Transfer,NaN,319.469427
115,0xab0cf426b4879324640b2fc41d86fbbaa240e1a87fa5...,16407586,1673729939,2023-01-14 20:58:59,0xa1c7fb8cb261ae26c0aefcd9edecba7860ba69c2,0x993f36f6bd7e92e1dc36faaecbe700df3534a34a,NaN,0.000000,3.096342,5100.356744,0.000395,0.649967,1550.07,NaN,NaN,Transfer,NaN,-4799.547102
